In [1]:
from math import prod

In [2]:
test_input_1 = '110100101111111000101000'
test_input_2 = '00111000000000000110111101000101001010010001001000000000'
test_input_3 = '11101110000000001101010000001100100000100011000001100000'
prod_input   = '020D78804D397973DB5B934D9280CC9F43080286957D9F60923592619D3230047C0109763976295356007365B37539ADE687F333EA8469200B666F5DC84E80232FC2C91B8490041332EB4006C4759775933530052C0119FAA7CB6ED57B9BBFBDC153004B0024299B490E537AFE3DA069EC507800370980F96F924A4F1E0495F691259198031C95AEF587B85B254F49C27AA2640082490F4B0F9802B2CFDA0094D5FB5D626E32B16D300565398DC6AFF600A080371BA12C1900042A37C398490F67BDDB131802928F5A009080351DA1FC441006A3C46C82020084FC1BE07CEA298029A008CCF08E5ED4689FD73BAA4510C009981C20056E2E4FAACA36000A10600D45A8750CC8010989716A299002171E634439200B47001009C749C7591BD7D0431002A4A73029866200F1277D7D8570043123A976AD72FFBD9CC80501A00AE677F5A43D8DB54D5FDECB7C8DEB0C77F8683005FC0109FCE7C89252E72693370545007A29C5B832E017CFF3E6B262126E7298FA1CC4A072E0054F5FBECC06671FE7D2C802359B56A0040245924585400F40313580B9B10031C00A500354009100300081D50028C00C1002C005BA300204008200FB50033F70028001FE60053A7E93957E1D09940209B7195A56BCC75AE7F18D46E273882402CCD006A600084C1D8ED0E8401D8A90BE12CCF2F4C4ADA602013BC401B8C11360880021B1361E4511007609C7B8CA8002DC32200F3AC01698EE2FF8A2C95B42F2DBAEB48A401BC5802737F8460C537F8460CF3D953100625C5A7D766E9CB7A39D8820082F29A9C9C244D6529C589F8C693EA5CD0218043382126492AD732924022CE006AE200DC248471D00010986D17A3547F200CA340149EDC4F67B71399BAEF2A64024B78028200FC778311CC40188AF0DA194CF743CC014E4D5A5AFBB4A4F30C9AC435004E662BB3EF0'

In [3]:
def hex_to_bin(hex):
    return ''.join([bin(int(h, 16)).split('0b')[1].zfill(4) for h in hex])

assert hex_to_bin('D2FE28') == '110100101111111000101000'
assert hex_to_bin('38006F45291200') == '00111000000000000110111101000101001010010001001000000000'

In [4]:
class Packet:
    
    def __init__(self, bits, debug=False):
        self.debug = debug
        self._print(f'bits: {bits}')
        self.bits = bits
        self.version = self._bin_to_int(bits[0:3])
        self.version_total = self.version
        self.type_id = self._bin_to_int(bits[3:6])
        self.index   = 6
        self.value   = None
        self._print(f'version: {self.version}')
        self._print(f'type_id: {self.type_id}')
        
        
        if self.type_id == 4:
            self.value = self._parse_literal()
            self._print(f'literal: {self.value}')
            return
        else:
            self.subpackets = []
            self._subpackets()
            self._do_math()
            self.version_total += sum([p.version_total for p in self.subpackets])
            return
        
    def _print(self, message):
        if self.debug:
            print(message)
    
    def _bin_to_int(self, bits):
        return int(f'0b{bits}', 2)
    
    def _do_math(self):
        values = [ packet.value for packet in self.subpackets if packet.value is not None ]
        if self.type_id == 0:
            self.value = sum(values)
        elif self.type_id == 1:
            self.value = prod(values)
        elif self.type_id == 2:
            self.value = min(values)
        elif self.type_id == 3:
            self.value = max(values)
        elif self.type_id == 5:
            self.value = 1 if values[0] > values[1] else 0
        elif self.type_id == 6:
            self.value = 1 if values[0] < values[1] else 0
        elif self.type_id == 7:
            self.value = 1 if values[0] == values[1] else 0
    
    def _parse_literal(self):
        value = 0
        little_bits = ''
        while True:
            little_bits += self.bits[self.index+1:self.index+5]
            self.index += 5
            if self.bits[self.index - 5] == '0':
                break
        return self._bin_to_int(little_bits)
        
    
    def _subpackets(self):
        length_type_id = self.bits[self.index]
        self._print(f'length type id: {length_type_id}')
        self.index += 1
        if length_type_id == '0':
            packet_length = self._bin_to_int(self.bits[self.index:self.index+15])
            self._print(f'packet length: {packet_length}')
            self.index += 15
            while packet_length > 0:
                sp = Packet(self.bits[self.index:])
                packet_length -= sp.index
                self.index += sp.index
                self.subpackets.append(sp)
        else:
            packet_count = self._bin_to_int(self.bits[self.index:self.index+11])
            self.index += 11
            while packet_count != 0:
                self._print(f'packet count: {packet_count}')
                sp = Packet(self.bits[self.index:])
                self.index += sp.index
                self.subpackets.append(sp)
                packet_count -= 1
                
p = Packet(hex_to_bin(prod_input))
p.version_total, p.value

(901, 110434737925)